In [1]:
!pip install filterpy
!pip install flask
!pip install flask_ngrok

     |████████████████████████████████| 177 kB 5.4 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=89a8144f2a5499a416778bf2792343263675c4a4e00eb682a82816eb5f960511
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy


In [2]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-11-12 13:21:20--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  63.5MB/s    in 4.0s    

2021-11-12 13:21:25 (59.5 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)
import json
from dataclasses import dataclass
import time
from __future__ import print_function
from numba import jit
import numpy as np
from sklearn.utils.linear_assignment_ import linear_assignment
from filterpy.kalman import KalmanFilter
import datetime
   # import the necessary packages
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import glob
import math
@app.route("/")
def home():

  
  @dataclass
  class State:
      """Data Class we'll use for creating events"""
      event: int
      timestamp: float
      location: str
      image: list
      
      def show_object(self):
        print(self.event, self.timestamp, self.location, self.image)

      def toJSON(self):
          return json.dumps(self, default=lambda o: o.__dict__)

      def convert_timestamp(self):
        time.mktime(datetime.datetime.strptime(self.timestamp,
                                        "%d/%m/%Y").timetuple())
  """
      SORT: A Simple, Online and Realtime Tracker
      Copyright (C) 2016 Alex Bewley alex@dynamicdetection.com

      This program is free software: you can redistribute it and/or modify
      it under the terms of the GNU General Public License as published by
      the Free Software Foundation, either version 3 of the License, or
      (at your option) any later version.

      This program is distributed in the hope that it will be useful,
      but WITHOUT ANY WARRANTY; without even the implied warranty of
      MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
      GNU General Public License for more details.

      You should have received a copy of the GNU General Public License
      along with this program.  If not, see <http://www.gnu.org/licenses/>.
  """


  @jit
  def iou(bb_test,bb_gt):
    """
    Computes IUO between two bboxes in the form [x1,y1,x2,y2]
    """
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1])
      + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
    return(o)

  def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
      the aspect ratio
    """
    w = bbox[2]-bbox[0]
    h = bbox[3]-bbox[1]
    x = bbox[0]+w/2.
    y = bbox[1]+h/2.
    s = w*h    #scale is just area
    r = w/float(h)
    return np.array([x,y,s,r]).reshape((4,1))

  def convert_x_to_bbox(x,score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
      [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2]*x[3])
    h = x[2]/w
    if(score==None):
      return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
    else:
      return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))

  class KalmanBoxTracker(object):
    """
    This class represents the internel state of individual tracked objects observed as bbox.
    """
    count = 0
    def __init__(self,bbox):
      """
      Initialises a tracker using initial bounding box.
      """
      #define constant velocity model
      self.kf = KalmanFilter(dim_x=7, dim_z=4)
      self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
      self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

      self.kf.R[2:,2:] *= 10.
      self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
      self.kf.P *= 10.
      self.kf.Q[-1,-1] *= 0.01
      self.kf.Q[4:,4:] *= 0.01

      self.kf.x[:4] = convert_bbox_to_z(bbox)
      self.time_since_update = 0
      self.id = KalmanBoxTracker.count
      KalmanBoxTracker.count += 1
      self.history = []
      self.hits = 0
      self.hit_streak = 0
      self.age = 0

    def update(self,bbox):
      """
      Updates the state vector with observed bbox.
      """
      self.time_since_update = 0
      self.history = []
      self.hits += 1
      self.hit_streak += 1
      self.kf.update(convert_bbox_to_z(bbox))

    def predict(self):
      """
      Advances the state vector and returns the predicted bounding box estimate.
      """
      if((self.kf.x[6]+self.kf.x[2])<=0):
        self.kf.x[6] *= 0.0
      self.kf.predict()
      self.age += 1
      if(self.time_since_update>0):
        self.hit_streak = 0
      self.time_since_update += 1
      self.history.append(convert_x_to_bbox(self.kf.x))
      return self.history[-1]

    def get_state(self):
      """
      Returns the current bounding box estimate.
      """
      return convert_x_to_bbox(self.kf.x)

  def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
    """
    Assigns detections to tracked object (both represented as bounding boxes)

    Returns 3 lists of matches, unmatched_detections and unmatched_trackers
    """
    if(len(trackers)==0) or (len(detections)==0):
      return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
    iou_matrix = np.zeros((len(detections),len(trackers)),dtype=np.float32)

    for d,det in enumerate(detections):
      for t,trk in enumerate(trackers):
        iou_matrix[d,t] = iou(det,trk)
    matched_indices = linear_assignment(-iou_matrix)

    unmatched_detections = []
    for d,det in enumerate(detections):
      if(d not in matched_indices[:,0]):
        unmatched_detections.append(d)
    unmatched_trackers = []
    for t,trk in enumerate(trackers):
      if(t not in matched_indices[:,1]):
        unmatched_trackers.append(t)

    #filter out matched with low IOU
    matches = []
    for m in matched_indices:
      if(iou_matrix[m[0],m[1]]<iou_threshold):
        unmatched_detections.append(m[0])
        unmatched_trackers.append(m[1])
      else:
        matches.append(m.reshape(1,2))
    if(len(matches)==0):
      matches = np.empty((0,2),dtype=int)
    else:
      matches = np.concatenate(matches,axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)

  class Sort(object):
    def __init__(self,max_age=1,min_hits=3):
      """
      Sets key parameters for SORT
      """
      self.max_age = max_age
      self.min_hits = min_hits
      self.trackers = []
      self.frame_count = 0

    def update(self,dets):
      """
      Params:
        dets - a numpy array of detections in the format [[x,y,w,h,score],[x,y,w,h,score],...]
      Requires: this method must be called once for each frame even with empty detections.
      Returns the a similar array, where the last column is the object ID.

      NOTE: The number of objects returned may differ from the number of detections provided.
      """
      
      # prevent "too many indices for array" error
      if len(dets)==0:
        return np.empty((0,5))

      self.frame_count += 1
      #get predicted locations from existing trackers.
      trks = np.zeros((len(self.trackers),5))
      to_del = []
      ret = []
      for t,trk in enumerate(trks):
        pos = self.trackers[t].predict()[0]
        trk[:] = [pos[0], pos[1], pos[2], pos[3], 0,]
        if(np.any(np.isnan(pos))):
          to_del.append(t)
      trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
      for t in reversed(to_del):
        self.trackers.pop(t)
      matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks)

      #update matched trackers with assigned detections
      for t,trk in enumerate(self.trackers):
        if(t not in unmatched_trks):
          d = matched[np.where(matched[:,1]==t)[0],0]
          trk.update(dets[d,:][0])

      #create and initialise new trackers for unmatched detections
      for i in unmatched_dets:
          trk = KalmanBoxTracker(dets[i,:])
          self.trackers.append(trk)
      i = len(self.trackers)
      for trk in reversed(self.trackers):
          d = trk.get_state()[0]
          if((trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits)):
            ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
          i -= 1
          #remove dead tracklet
          if(trk.time_since_update > self.max_age):
            self.trackers.pop(i)
      if(len(ret)>0):
        return np.concatenate(ret)
      return np.empty((0,5))
    
 

  files = glob.glob('output/*.png')
  for f in files:
      os.remove(f)

  tracker = Sort()
  memory = {}
  #line1 = [(400,638), (1250, 788)]
  #line2 = [(120,838), (1120, 1080)]
  #counter1 = 0
  #counter2 = 0

  # construct the argument parse and parse the arguments

  arg_confi = 0.35
  arg_thres = 0.25
  def adjust_gamma(image, gamma=1.0):
      # build a lookup table mapping the pixel values [0, 255] to
      # their adjusted gamma values
      invGamma = 1.0 / gamma
      table = np.array([((i / 255.0) ** invGamma) * 255
          for i in np.arange(0, 256)]).astype("uint8")
  
      # apply gamma correction using the lookup table
      return cv2.LUT(image, table)

  # Return true if line segments AB and CD intersect
  #def intersect(A,B,C,D):
  #    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

  #def ccw(A,B,C):
  #    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

  # load the COCO class labels our YOLO model was trained on
  labelsPath = os.path.sep.join(["coco.names"])
  LABELS = open(labelsPath).read().strip().split("\n")

  # initialize a list of colors to represent each possible class label
  np.random.seed(42)
  COLORS = np.random.randint(0, 255, size=(200, 3),
      dtype="uint8")

  # derive the paths to the YOLO weights and model configuration
  weightsPath = os.path.sep.join(["yolov3.weights"])
  configPath = os.path.sep.join(["yolov3.cfg"])

  # load our YOLO object detector trained on COCO dataset (80 classes)
  # and determine only the *output* layer names that we need from YOLO
  print("[INFO] loading YOLO from disk...")
  net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
  ln = net.getLayerNames()
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

  # initialize the video stream, pointer to output video file, and
  # frame dimensions
  vs = cv2.VideoCapture('cars3.mp4')
  writer = None
  (W, H) = (None, None)

  frameIndex = 0
  # try to determine the total number of frames in the video file



  # Adds to unnecesary processing, just used for testing purpose, to be commented in final code.
  #start
  try:
      prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
          else cv2.CAP_PROP_FRAME_COUNT
      total = int(vs.get(prop))
      print("[INFO] {} total frames in video".format(total))
      
  # an error occurred while trying to determine the total
  # number of frames in the video file
  except:
      print("[INFO] could not determine # of frames in video")
      print("[INFO] no approx. completion time can be provided")
      total = -1

  #end


  # loop over frames from the video file stream
  while True:
      # read the next frame from the file
      (grabbed, frame) = vs.read()

      # if the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
          break

      # if the frame dimensions are empty, grab them
      if W is None or H is None:
          (H, W) = frame.shape[:2]

      frame = adjust_gamma(frame, gamma=1.5)
      # construct a blob from the input frame and then perform a forward
      # pass of the YOLO object detector, giving us our bounding boxes
      # and associated probabilities
      blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (256, 256),
          swapRB=True, crop=False)
      net.setInput(blob)
      start = time.time()
      layerOutputs = net.forward(ln)
      end = time.time()

      # initialize our lists of detected bounding boxes, confidences,
      # and class IDs, respectively
      boxes = []
      center = []
      confidences = []
      classIDs = []

      # loop over each of the layer outputs
      for output in layerOutputs:
          # loop over each of the detections
          for detection in output:
              # extract the class ID and confidence (i.e., probability)
              # of the current object detection
              scores = detection[5:]
              classID = np.argmax(scores)
              confidence = scores[classID]

              # filter out weak predictions by ensuring the detected
              # probability is greater than the minimum probability
              if confidence > arg_confi:
                  # scale the bounding box coordinates back relative to
                  # the size of the image, keeping in mind that YOLO
                  # actually returns the center (x, y)-coordinates of
                  # the bounding box followed by the boxes' width and
                  # height
                  box = detection[0:4] * np.array([W, H, W, H])
                  (centerX, centerY, width, height) = box.astype("int")
                  
                  # use the center (x, y)-coordinates to derive the top
                  # and and left corner of the bounding box
                  x = int(centerX - (width / 2))
                  y = int(centerY - (height / 2))

                  # update our list of bounding box coordinates,
                  # confidences, and class IDs
                  center.append(int(centerY))
                  boxes.append([x, y, int(width), int(height)])
                  confidences.append(float(confidence))
                  classIDs.append(classID)
                  

      # apply non-maxima suppression to suppress weak, overlapping
      # bounding boxes
      idxs = cv2.dnn.NMSBoxes(boxes, confidences, arg_confi, arg_thres)
      #print("idxs", idxs)
      #print("boxes", boxes[i][0])
      #print("boxes", boxes[i][1])
      
      dets = []
      if len(idxs) > 0:
          # loop over the indexes we are keeping
          for i in idxs.flatten():
              (x, y) = (boxes[i][0], boxes[i][1])
              (w, h) = (boxes[i][2], boxes[i][3])
              dets.append([x, y, x+w, y+h, confidences[i]])
              #print(confidences[i])
              #print(center[i])
      np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
      dets = np.asarray(dets)
      tracks = tracker.update(dets)
      
      boxes = []
      indexIDs = []
      c = []
      
      previous = memory.copy()
      #print("centerx",centerX)
      #  print("centery",centerY)
      memory = {}

      for track in tracks:
          boxes.append([track[0], track[1], track[2], track[3]])
          indexIDs.append(int(track[4]))
          memory[indexIDs[-1]] = boxes[-1]

      if len(boxes) > 0:
          i = int(0)
          step_rate = 1
          for box in boxes[::step_rate]:
              # extract the bounding box coordinates
              (x, y) = (int(box[0]), int(box[1]))
              (w, h) = (int(box[2]), int(box[3]))

              # draw a bounding box rectangle and label on the image
              # color = [int(c) for c in COLORS[classIDs[i]]]
              # cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

              color = [int(c) for c in COLORS[indexIDs[i] % len(COLORS)]]
              cv2.rectangle(frame, (x, y), (w, h), color, 2)

              if indexIDs[i] in previous:
                  previous_box = previous[indexIDs[i]]
                  (x2, y2) = (int(previous_box[0]), int(previous_box[1]))
                  (w2, h2) = (int(previous_box[2]), int(previous_box[3]))
                  p0 = (int(x + (w-x)/2), int(y + (h-y)/2))
                  p1 = (int(x2 + (w2-x2)/2), int(y2 + (h2-y2)/2))
                  #cv2.line(frame, p0, p1, color, 3)
                  
                  #Speed Calculation
                  y_pix_dist = int(y + (h-y)/2) - int(y2 + (h2-y2)/2)
                  text_y = "{} y".format(y_pix_dist)
                  x_pix_dist = int(x + (w-x)/2) - int(x2 + (w2-x2)/2)
                  text_x = "{} x".format(x_pix_dist)
                  #cv2.putText(frame, text_y, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 4)
                  #cv2.putText(frame, text_x, (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 4)
                  
                  ppm = 5*step_rate*(math.sqrt(abs(y2/H)))
                  
                  final_pix_dist = math.sqrt(abs(y_pix_dist*y_pix_dist)+abs(x_pix_dist*x_pix_dist))
                  speed = abs(np.round(y_pix_dist * ppm,2))
                  
                  #speed = np.round(final_pix_dist*60*ppm)
                  text_speed = "{} km/h".format(speed)
                  cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                  #if frame, para1, para2
                  if frameIndex%30 == 0:
                    if abs(speed)<=2.5 and speed >= -2.5:
                      text_speed = "{} km/h, Stopped vehicle".format(speed)
                      event = 1
                      location = "cars.mp4"
                      current_time = datetime.datetime.now()
                      image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                      E = State(event, current_time, location, image)
                      print(E.toJSON)
                      #event id 1
                      #json created 
                    else:
                        if x2 < W/2:    
                          #origin -2.5 -> 0
                          # -1.5 > -2.5 
                          # -3.5 < -2.5
                            if speed < -2.5 and abs(speed)>0 and abs(speed)<20:
                                text_speed = "{} km/h, Slow-Moving Reverse-Moving vehicle".format(speed)
                                event = 2
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if speed > -2.5 and abs(speed)>0 and abs(speed)<20:
                                text_speed = "{} km/h, Slow-Moving vehicle".format(speed)
                                event = 3
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if speed < -2.5 and abs(speed) > 20:
                                text_speed = "{} km/h, Reverse-Moving Vehicle"
                                event = 4
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if abs(speed)>20:
                                text_speed = "{} km/h".format(speed)
                        else:
                            if speed*(-1) < -2.5 and abs(speed)>0 and abs(speed)<20:
                                text_speed = "{} km/h, Slow-Moving Reverse-Moving vehicle".format(abs(speed))
                                event = 1
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if speed*(-1) > -2.5 and abs(speed)>0 and abs(speed)<20:
                                text_speed = "{} km/h, Slow-Moving vehicle".format(abs(speed))
                                event = 1
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if speed*(-1) < -2.5 and abs(speed) > 20:
                                text_speed = "{} km/h, Reverse-Moving Vehicle"
                                event = 1
                                location = "cars.mp4"
                                current_time = datetime.datetime.now()
                                image = cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                                E = State(event, current_time, location, image)
                                print(E.toJSON)
                            if abs(speed)>20:
                                text_speed = "{} km/h".format(abs(speed))
                      
                  cv2.putText(frame, text_speed, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                  #if intersect(p0, p1, line1[0], line1[1]):
                      #counter1 += 1
  #                if intersect(p0, p1, line2[0], line2[1]):
  #                    counter2 += 1

              # text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
              #text = "{}".format(indexIDs[i])
              #cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
              i += 1

      # draw line
      #cv2.line(frame, line1[0], line1[1], (0, 255, 255), 4)
  #    cv2.line(frame, line2[0], line2[1], (255, 0, 255), 2)

      #note_text = "NOTE: Vehicle speeds are calibrated only at yellow line. speed of cars are more stable."
      #cv2.putText(frame, note_text, (50,110), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 2)
      # draw counter
      #counter_text = "counter:{}".format(counter1)
      #cv2.putText(frame, counter_text, (100,250), cv2.FONT_HERSHEY_DUPLEX, 4.0, (0, 0, 255), 7)
      #cv2.putText(frame, "ctr2",str(counter2), (100,400), cv2.FONT_HERSHEY_DUPLEX, 5.0, (255, 0, 255), 10)
      # counter += 1

      # saves image file every nth frame
      if frameIndex%4==0:
          cv2.imwrite("Images/frame-{}.png".format(frameIndex), frame)

      # check if the video writer is None
      if writer is None:
          # initialize our video writer
          fourcc = cv2.VideoWriter_fourcc(*"XVID")
          writer = cv2.VideoWriter('output.mp4', fourcc, 15,
              (frame.shape[1], frame.shape[0]), True)

          # some information on processing single frame
          if total > 0:
              elap = (end - start)
              print("[INFO] single frame took {:.4f} seconds".format(elap))
              print("[INFO] estimated total time to finish: {:.4f}".format(
                  elap * total))

      # write the output frame to disk
      writer.write(frame)

      # increase frame index
      frameIndex += 1

      #if frameIndex >= 4000: # limits the execution to the first 4000 frames
      #    print("[INFO] cleaning up...")
      #    writer.release()
      #    vs.release()
      #    exit()

  # release the file pointers
  print("[INFO] cleaning up...")
  writer.release()
  vs.release()
 
if __name__ == "__main__":
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://84d6-35-226-247-73.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
[INFO] loading YOLO from disk...
[INFO] 466 total frames in video
[INFO] single frame took 0.9576 seconds
[INFO] estimated total time to finish: 446.2394


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


<bound method home.<locals>.State.toJSON of home.<locals>.State(event=1, timestamp=datetime.datetime(2021, 11, 12, 13, 31, 20, 969850), location='cars.mp4', image=array([[[ 81, 144, 115],
        [ 71, 137, 107],
        [ 86, 140, 115],
        ...,
        [ 57,  62,  45],
        [ 57,  62,  45],
        [ 62,  67,  51]],

       [[ 87, 149, 120],
        [ 86, 148, 119],
        [106, 156, 133],
        ...,
        [ 57,  62,  45],
        [ 54,  59,  41],
        [ 59,  65,  48]],

       [[ 95, 153, 130],
        [ 89, 149, 125],
        [104, 156, 134],
        ...,
        [ 65,  69,  55],
        [ 63,  67,  54],
        [ 62,  66,  52]],

       ...,

       [[148, 142, 138],
        [146, 141, 136],
        [146, 141, 136],
        ...,
        [131, 129, 120],
        [136, 134, 126],
        [138, 136, 128]],

       [[148, 142, 138],
        [146, 141, 136],
        [146, 141, 136],
        ...,
        [135, 133, 125],
        [136, 134, 126],
        [135, 133, 125]],
